In [1]:
import os
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
from IPython.display import clear_output

In [2]:
DATASET_DIR = "data/anime_face"

IMAGE_DIM = 64
BATCH_SIZE = 128
CODE_SIZE = 100
RGB_CHANNEL_COUNT = 3

LEARNING_RATE = 0.001
BETA = 0.5
EPOCHS = 2000

CHECKPOINTS_FOLDER = "checkpoints"


GEN_FOLDER = "gen"
DISC_FOLDER = "disc"

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "data/anime_face",
    label_mode=None,
    image_size=(IMAGE_DIM, IMAGE_DIM),
    batch_size=BATCH_SIZE,
)
dataset = dataset.map(lambda x: (x / 127.5) - 1)
dataset = dataset.shuffle(buffer_size=100)


Found 63565 files belonging to 1 classes.
Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-03-21 01:40:03.081531: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-21 01:40:03.081683: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
generator = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(CODE_SIZE,)),
        tf.keras.layers.Dense(units=4 * 4 * 256),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Reshape(target_shape=(4, 4, 256)),
        tf.keras.layers.Conv2DTranspose(
            filters=128, kernel_size=4, strides=2, padding="same"
        ),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2DTranspose(
            filters=64, kernel_size=4, strides=2, padding="same"
        ),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2DTranspose(
            32, kernel_size=4, strides=2, padding="same"
        ),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2DTranspose(
            3, kernel_size=4, strides=2, padding="same"
        ),
        tf.keras.layers.Activation("tanh"),
    ]
)


In [5]:
image_shape = (IMAGE_DIM, IMAGE_DIM, RGB_CHANNEL_COUNT)
discriminator = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(
            32,
            kernel_size=3,
            strides=2,
            input_shape=image_shape,
            padding="same",
        ),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding="same"),
        tf.keras.layers.ZeroPadding2D(padding=((0, 1), (0, 1))),
        tf.keras.layers.BatchNormalization(momentum=0.8),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
        tf.keras.layers.BatchNormalization(momentum=0.8),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding="same"),
        tf.keras.layers.BatchNormalization(momentum=0.8),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1),
    ]
)


In [6]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

generator_optimizer = tf.keras.optimizers.Adagrad(
    learning_rate=LEARNING_RATE,
    initial_accumulator_value=0.01,
)
discriminator_optimizer = tf.keras.optimizers.Adagrad(
    learning_rate=LEARNING_RATE,
    initial_accumulator_value=0.01,
)

""" generator_optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE, beta_1=BETA
)
discriminator_optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE, beta_1=BETA
)
 """

def discriminator_loss(real, fake):
    real_loss = loss(tf.ones_like(real), real)
    fake_loss = loss(tf.zeros_like(fake), fake)
    return real_loss + fake_loss


def generator_loss(fake):
    gen_loss = loss(tf.ones_like(fake), fake)
    return gen_loss


@tf.function
def train_step(images):
    noise = tf.random.normal([256, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gen_grad = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_grad = disc_tape.gradient(
        disc_loss, discriminator.trainable_variables
    )

    generator_optimizer.apply_gradients(
        zip(gen_grad, generator.trainable_variables)
    )
    discriminator_optimizer.apply_gradients(
        zip(disc_grad, discriminator.trainable_variables)
    )

    return gen_loss, disc_loss


In [7]:
def train(dataset, epochs):
    gen_loss = 0
    disc_loss = 0

    gen_plot = []
    disc_plot = []

    for epoch in range(epochs):
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        gen_plot.append(gen_loss)
        disc_plot.append(disc_loss)

        if epoch % 25 == 0:
            OUTPUT_FOLDER = os.path.join(CHECKPOINTS_FOLDER, f"adagrad_epoch{epoch}-of-{EPOCHS}_lr{LEARNING_RATE}_beta{BETA}")
            GEN_CHECKPOINT_FOLDER = os.path.join(OUTPUT_FOLDER, GEN_FOLDER)
            DISC_CHECKPOINT_FOLDER = os.path.join(OUTPUT_FOLDER, DISC_FOLDER)

            generator.save(GEN_CHECKPOINT_FOLDER)
            discriminator.save(DISC_CHECKPOINT_FOLDER)

            noise = np.random.rand(32,100)
            pred = generator.predict(noise)
            tf.keras.preprocessing.image.save_img(os.path.join(OUTPUT_FOLDER, "sample.png"), pred[0,:,:])
            display(tf.keras.preprocessing.image.array_to_img(pred[0]))

            clear_output(wait=True)

            x = [X for X in range(epoch+1)]
            plt.plot(x, gen_plot)
            plt.plot(x, disc_plot)
            plt.title("epoch vs loss")
            plt.legend(["gen_loss", "dis_loss"])
            plt.xlabel("epochs")
            plt.ylabel("loss")
            plt.show()

        print(
            f"epoch: {epoch}/{epochs}; G_loss: {gen_loss:.6f}; D_loss: {disc_loss:.6f}"
        )


In [8]:
train(dataset, EPOCHS)

2022-03-21 01:40:09.249276: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-21 01:40:10.393174: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
noise = np.random.rand(32,100)
pred = generator.predict(noise)

2022-03-21 00:32:01.489059: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
w=10
h=10
fig=plt.figure(figsize=(8, 8))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(pred[i])
plt.show()